In [ ]:
import ase.io as ase_io
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_rmse(a, b, perc=False):
    rmse = np.sqrt(np.mean(np.square(a - b)))
    if perc:
        return 100 * rmse / b.std(ddof=1)
    return rmse

In [ ]:
# define colors

c0 = "#f18f01"
c1 = "#033f63"
c2 = "#95b46a"
c3 = "#ee4266"

In [ ]:
# load rMD17 trajectories
mols = ["aspirin", "ethanol", "malonaldehyde", "naphthalene", "salicylic", "toluene"]
md17_frames = {}
for mol in mols:
    md17_frames[mol] = ase_io.read(f"../../data/rMD17/{mol}.xyz", ":")

we take the energy of the first molecule as a reference to mitigate against the differences in QM theory levels


In [ ]:
rmd17_castep_energy_err = []
for mol in mols:
    a = np.array([x.info["rMD17_energy"] for x in md17_frames[mol]])
    a -= a[0]
    a = a[1:]
    b = np.array([x.info["castep_energy"] for x in md17_frames[mol]])
    b -= b[0]
    b = b[1:]
    print(mol)
    print(1e3 * get_rmse(a, b) / len(md17_frames[mol][0]))
    print()
    rmd17_castep_energy_err.append(1e3 * get_rmse(a, b) / len(md17_frames[mol][0]))

In [ ]:
rmd17_castep_forces_err = []
for mol in mols:
    a = np.array([x.arrays["rMD17_forces"] for x in md17_frames[mol]])
    # a -= a[0]
    b = np.array([x.arrays["castep_forces"] for x in md17_frames[mol]])
    # b -= b[0]
    print(mol)
    print(1e3 * get_rmse(a, b))
    print()
    rmd17_castep_forces_err.append(1e3 * get_rmse(a, b))

In [ ]:
rmd17_mace_energy_err = []
for mol in mols:
    a = np.array([x.info["rMD17_energy"] for x in md17_frames[mol]])
    a -= a[0]
    a = a[1:]
    b = np.array([x.info["gomace_energy"] for x in md17_frames[mol]])
    b -= b[0]
    b = b[1:]
    print(mol)
    print(1e3 * get_rmse(a, b) / len(md17_frames[mol][0]))
    print()
    rmd17_mace_energy_err.append(1e3 * get_rmse(a, b) / len(md17_frames[mol][0]))

In [ ]:
rmd17_mace_forces_err = []
for mol in mols:
    a = np.array([x.arrays["rMD17_forces"] for x in md17_frames[mol]])
    # a -= a[0]
    b = np.array([x.arrays["gomace_forces"] for x in md17_frames[mol]])
    # b -= b[0]
    print(mol)
    print(1e3 * get_rmse(a, b))
    print()
    rmd17_mace_forces_err.append(1e3 * get_rmse(a, b))

In [ ]:
# change how the long molecule names are displayed
new_mols = mols.copy()
new_mols[2] = "malo."
new_mols[3] = "naphth."

In [ ]:
fig = plt.figure(figsize=(3.5, 3.0), constrained_layout=True)

ax = fig.add_subplot(211)
ax.bar(
    np.arange(len(new_mols)),
    rmd17_mace_energy_err,
    width=0.3,
    label="GO-MACE-23 prediction",
    color=c3,
    edgecolor=c3,
)
ax.bar(
    np.arange(len(new_mols)),
    rmd17_castep_energy_err,
    width=0.3,
    hatch="///",
    edgecolor="k",
    fill=False,
    lw=0.75,
    label="GO-MACE-23 reference",
)
ax.plot([-0.1, 5.1], [1.8] * 2, "k--", lw=0.75, label="GO-MACE-23 validation")
ax.set_xticklabels(())
ax.tick_params(axis="both", labelsize=8)
ax.set_ylabel("Energy RMSE (meV at.$^{-1}$)", fontsize=8)
ax.legend(loc="upper right", fontsize=8, ncols=1, frameon=False)
ax.set_ylim(0, 16)

labelx = -0.11
ax.yaxis.set_label_coords(labelx, 0.5, transform=ax.transAxes)

ax = fig.add_subplot(212)
ax.bar(
    np.arange(len(new_mols)), rmd17_mace_forces_err, width=0.3, color=c3, edgecolor=c3
)
ax.bar(
    np.arange(len(new_mols)),
    rmd17_castep_forces_err,
    width=0.3,
    hatch="///",
    edgecolor="k",
    fill=False,
    lw=0.75,
)

# GO-MACE-23 validation errors on the GO dataset
ax.plot([-0.1, 5.1], [106.9] * 2, "k--", lw=0.75)

ax.set_xticks(np.arange(len(new_mols)))
ax.set_xticklabels(new_mols)
ax.tick_params(axis="both", labelsize=8)
ax.set_ylabel("Forces RMSE (meV $\AA^{-1}$)", fontsize=8)
ax.yaxis.set_label_coords(labelx, 0.5, transform=ax.transAxes)

# fig.savefig("./fig2.svg", dpi=300)